In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-10-31 14:39:09.177252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 14:39:10.608288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0


2024-10-31 14:39:13.902428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-31 14:39:14.291364: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

import kagglehub

/home/jer/anaconda3/envs/cap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download dataset
path =  kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")
print("Path to dataset files:", path)

Path to dataset files: /home/jer/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1


In [4]:
p = path + '/Data/genres_original'

In [5]:
# Parameters
sr = 22050  # Sample rate
duration = 5  # Duration of each audio clip in seconds
samples_per_track = sr * duration

# Load data
def load_data(data_path):
    labels = []
    features = []

    for genre in os.listdir(data_path):
        genre_path = os.path.join(data_path, genre)
        if os.path.isdir(genre_path):
            for file in os.listdir(genre_path):
                if file.endswith('.wav'):
                    file_path = os.path.join(genre_path, file)
                    try:
                        audio, _ = librosa.load(file_path, sr=sr)
                        if len(audio) >= samples_per_track:
                            audio = audio[:samples_per_track]
                        else:
                            audio = np.pad(audio, (0, max(0, samples_per_track - len(audio))), "constant")
                        features.append(audio)
                        labels.append(genre)
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")
    
    return np.array(features), np.array(labels)

X, y = load_data(p)

/tmp/ipykernel_11221/3561210060.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=sr)
/home/jer/anaconda3/envs/cap/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error loading /home/jer/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1/Data/genres_original/jazz/jazz.00054.wav: 


In [6]:
X.shape

(999, 110250)

In [7]:

# Check shape of X
print("Original shape of features:", X.shape)  # Should be (999, 110250)

# Encode labels
unique_labels, y_encoded = np.unique(y, return_inverse=True)
y_categorical = to_categorical(y_encoded)

# Reshape features for Conv1D
# Reshape to (num_samples, samples_per_track, 1)
X = X.reshape(X.shape[0], samples_per_track, 1)  # Add channel dimension

# Check new shape
print("New shape of features for Conv1D:", X.shape)  # Should be (999, 110250, 1)

Original shape of features: (999, 110250)
New shape of features for Conv1D: (999, 110250, 1)


In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Change to the desired GPU if needed

In [9]:
from keras.layers import Input

def create_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),  # Use Input layer
        Conv1D(32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Output layer
    ])
    return model


In [10]:
# Correct input shape for Conv1D
input_shape = (samples_per_track, 1)  # (timesteps, channels)
num_classes = len(unique_labels)  # Number of music genres

# Create the model
model = create_model(input_shape, num_classes)

2024-10-31 14:39:24.950000: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 903118848 exceeds 10% of free system memory.
2024-10-31 14:39:25.174190: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 903118848 exceeds 10% of free system memory.
2024-10-31 14:39:25.389116: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 903118848 exceeds 10% of free system memory.


In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 110248, 32)        128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 55124, 32)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 55122, 64)         6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 27561, 64)        0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 1763904)           0         
                                                                 
 dense (Dense)               (None, 128)               2

In [12]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=16)

Epoch 1/2


2024-10-31 14:39:26.384299: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 903118848 exceeds 10% of free system memory.
2024-10-31 14:39:26.521299: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 903118848 exceeds 10% of free system memory.


50/50 [==============================] - 199s 4s/step - loss: 3.9257 - accuracy: 0.1740 - val_loss: 2.1710 - val_accuracy: 0.2000
Epoch 2/2
50/50 [==============================] - 196s 4s/step - loss: 1.9769 - accuracy: 0.3066 - val_loss: 2.0570 - val_accuracy: 0.2200


In [13]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

7/7 [==============================] - 5s 650ms/step - loss: 2.0570 - accuracy: 0.2200
Validation Loss: 2.0570, Validation Accuracy: 0.2200


In [14]:
# Make predictions on the validation set
predictions = model.predict(X_val)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_val, axis=1)

# Display some predictions
for i in range(5):
    print(f'True class: {unique_labels[true_classes[i]]}, Predicted class: {unique_labels[predicted_classes[i]]}')

# Save the model
model.save('music_genre_classification_model.h5')

7/7 [==============================] - 5s 692ms/step
True class: disco, Predicted class: hiphop
True class: reggae, Predicted class: rock
True class: jazz, Predicted class: jazz
True class: pop, Predicted class: hiphop
True class: reggae, Predicted class: country
